# RFM Customer Analysis Demo

This demo will show how simple it is to calculate RFM scores. Let's use Microsoft's AdventureWorks international sales data for the demo.

Thanks to Susan Li (https://towardsdatascience.com/@actsusanli) for laying out some of the ground work.

* [Brief RFM Description](#description)
* [Load AdventureWorks](#load)
* [Data Manipulation](#manipulation)
* [Define RFM Scores](#scores)
* [Define Quantiles](#quantiles)
* [Calculations](#calculations)
* [Display Results](#results)

### Brief RFM description
<a name="description"> </a>

Accessed from Wikipedia on 4/5/2018.

https://en.wikipedia.org/wiki/RFM_(customer_value)

RFM is a method used for analyzing customer value. It is commonly used in database
marketing and direct marketing and has received particular attention in retail and
professional services industries.

RFM stands for the three dimensions:

1. Recency – How recently did the customer purchase?
2. Frequency – How often do they purchase?
3. Monetary Value – How much do they spend?

First, the imports. Pandas is a fantastic library for manipulating data and analysis. NumPy is used for a data type conversion later on in the demo.

In [1]:
import numpy as np
import pandas as pd

### Load AdventureWorks
<a name="load"> </a>

The AdventureWorks CSV doesn't come with column names, so I manually created this short list after a quick Google search.

In [2]:
headers = ['ProductKey',
           'OrderDateKey',
           'DueDateKey',
           'ShipDateKey',
           'CustomerKey',
           'PromotionKey',
           'CurrencyKey',
           'SalesTerritoryKey',
           'SalesOrderNumber',
           'SalesOrderLineNumber',
           'RevisionNumber',
           'OrderQuantity',
           'UnitPrice',
           'ExtendedAmount',
           'UnitPriceDiscountPct',
           'DiscountAmount',
           'ProductStandardCost',
           'TotalProductCost',
           'SalesAmount',
           'TaxAmt',
           'Freight',
           'CarrierTrackingNumber',
           'CustomerPONumber',
           'SalesOrderDesc',
           'MissingName1',
           'MissingName2'
          ]

We can access the AdventureWorks data directly from Microsoft's GitHub and simultaneously apply our headers. Also, the last 2 column names seem to be missing, so we're just going to ignore those for the sake of the demo.

In [3]:
data = pd.read_csv(
    'https://raw.githubusercontent.com/Microsoft/sql-server-samples/master/samples/databases/adventure-works/data-warehouse-install-script/FactInternetSales.csv',
    sep='|', header=None, names=headers, encoding='utf-16')

Date data type conversion. I'm only going to bother changing the one date column that we need to use.

In [4]:
data['OrderDate'] = pd.to_datetime(data['OrderDateKey'].astype(str), format='%Y%m%d')

And here's what we're working with.

In [5]:
data.head()

,ProductKey,OrderDateKey,DueDateKey,ShipDateKey,CustomerKey,PromotionKey,CurrencyKey,SalesTerritoryKey,SalesOrderNumber,SalesOrderLineNumber,...,TotalProductCost,SalesAmount,TaxAmt,Freight,CarrierTrackingNumber,CustomerPONumber,SalesOrderDesc,MissingName1,MissingName2,OrderDate
0,310,20101229,20110110,20110105,21768,1,19,6,SO43697,1,...,2171.2942,3578.2700,286.2616,89.4568,NaN,NaN,2010-12-29 00:00:00,2011-01-10 00:00:00,2011-01-05 00:00:00,2010-12-29
1,346,20101229,20110110,20110105,28389,1,39,7,SO43698,1,...,1912.1544,3399.9900,271.9992,84.9998,NaN,NaN,2010-12-29 00:00:00,2011-01-10 00:00:00,2011-01-05 00:00:00,2010-12-29
2,346,20101229,20110110,20110105,25863,1,100,1,SO43699,1,...,1912.1544,3399.9900,271.9992,84.9998,NaN,NaN,2010-12-29 00:00:00,2011-01-10 00:00:00,2011-01-05 00:00:00,2010-12-29
3,336,20101229,20110110,20110105,14501,1,100,4,SO43700,1,...,413.1463,699.0982,55.9279,17.4775,NaN,NaN,2010-12-29 00:00:00,2011-01-10 00:00:00,2011-01-05 00:00:00,2010-12-29
4,346,20101229,20110110,20110105,11003,1,6,9,SO43701,1,...,1912.1544,3399.9900,271.9992,84.9998,NaN,NaN,2010-12-29 00:00:00,2011-01-10 00:00:00,2011-01-05 00:00:00,2010-12-29


### Data Manipulation
<a name="manipulation"> </a>

This is the date of the most recent sale in the AdventureWorks data. We're going to pretend this date is the current date and use that to calculate our customer's recency.

In [6]:
most_recent_date = data['OrderDate'].max()

Create a new table with the grain being the customer by aggregating the sales data.

Without looking into the AdventureWorks data too deeply, it appears that the sales amounts are stored in various currencies. For the sake of the demo, let's just pretend they're all stored as USD.

In [7]:
customer_table = data.groupby('CustomerKey').agg(
    {
        'OrderDate': lambda x: most_recent_date - x.max(),
        'SalesOrderNumber': lambda x: len(x),
        'SalesAmount': lambda x: x.sum()
    }
)

Let's use more appropriate column names.

In [8]:
customer_table.rename(columns=
    {
        'OrderDate': 'Days Since Last Order',
        'SalesOrderNumber': 'Total Number of Orders',
        'SalesAmount': 'Total Sales'
    },
    inplace=True
)
customer_table.head()

,Days Since Last Order,Total Number of Orders,Total Sales
CustomerKey,,,
11000,270 days,8,8248.99
11001,49 days,11,6383.88
11002,339 days,4,8114.04
11003,263 days,9,8139.29
11004,272 days,6,8196.01


### Define Quantiles
<a name='quantiles'> </a>

With an RFM calculation, you can use any scale for the 3 different dimensions. Let's go ahead and just use quartiles for this demo.

Unfortunately, the Pandas function for implementing quantiles doesn't play well with the time delta data type that we're currently using, so let's convert that to an integer.

In [9]:
customer_table['Days Since Last Order'] = (customer_table['Days Since Last Order'] / np.timedelta64(1, 'D')).astype(int)

quantiles = customer_table.quantile(q=[0.25, 0.5, 0.75])
quantiles

,Days Since Last Order,Total Number of Orders,Total Sales
0.25,86.0,2.0,49.970
0.50,168.0,3.0,270.265
0.75,263.0,4.0,2511.275


### Define Scores
<a name='scores'> </a>

Let's define a few functions so that we can use our quartiles to segregate the data. These functions could easily be made more generic, but we're just going to keep it simple for the demo.

In [10]:
def recency_score(x, p, d):
    if x <= d[p][0.25]:
            return 1
    elif x <= d[p][0.50]:
            return 2
    elif x <= d[p][0.75]:
        return 3
    else:
        return 4
    
def frequency_and_monetary_score(x, p, d):
    if x <= d[p][0.25]:
        return 4
    elif x <= d[p][0.5]:
        return 3
    elif x <= d[p][0.75]:
        return 2
    else:
        return 1

### Calculations
<a name='calculations'> </a>

Calculate the recency quartile/score.

In [11]:
customer_table['Recency Quartile'] = \
    customer_table['Days Since Last Order'].apply(
        recency_score, args=('Days Since Last Order', quantiles))

Calculate the frequency quartile/score.

In [12]:
customer_table['Frequency Quartile'] = \
    customer_table['Total Number of Orders'].apply(
        frequency_and_monetary_score, args=('Total Number of Orders', quantiles))

Calculate the monetary quartile/score.

In [13]:
customer_table['Monetary Quartile'] = \
    customer_table['Total Sales'].apply(
        frequency_and_monetary_score, args=('Total Sales', quantiles))

### Display Results
<a name='results'> </a>

Let's take the individual scores and display them as a joined string, which is fairly common practice for RFM scoring.

In [14]:
customer_table['RFM Score'] = customer_table['Recency Quartile'].map(str) \
    + customer_table['Frequency Quartile'].map(str) \
    + customer_table['Monetary Quartile'].map(str)
    
customer_table.head()

,Days Since Last Order,Total Number of Orders,Total Sales,Recency Quartile,Frequency Quartile,Monetary Quartile,RFM Score
CustomerKey,,,,,,,
11000,270,8,8248.99,4,1,1,411
11001,49,11,6383.88,1,1,1,111
11002,339,4,8114.04,4,2,1,421
11003,263,9,8139.29,3,1,1,311
11004,272,6,8196.01,4,1,1,411


Sort best customers (RFM == '111') by total sales. These are arguably AdventureWork's 10 best customers (again, assuming all sales are listed in USD).

In [15]:
customer_table[customer_table['RFM Score'] == '111'].sort_values('Total Sales', ascending=False).head(10)

,Days Since Last Order,Total Number of Orders,Total Sales,Recency Quartile,Frequency Quartile,Monetary Quartile,RFM Score
CustomerKey,,,,,,,
12301,69,13,13295.3800,1,1,1,111
12308,70,14,13269.2700,1,1,1,111
12321,68,15,13215.6500,1,1,1,111
12307,72,11,13173.1900,1,1,1,111
11417,44,17,11248.4582,1,1,1,111
11420,60,17,11200.7696,1,1,1,111
11242,55,12,11068.0082,1,1,1,111
11245,48,9,10580.3500,1,1,1,111
11246,39,7,10575.3300,1,1,1,111


These scores could be loaded into a variety of tools to create some great visuals. We could also implement some visuals in Python using matplotlib. We'll see if I get around to doing so.